In [ ]:
from estival.wrappers import nevergrad as eng
from autumn.projects.sm_covid2.common_school.calibration import get_bcm_object
from autumn.projects.sm_covid2.common_school.calibration_plots.opti_plots import plot_model_fit
import nevergrad as ng

In [ ]:
opt_class = ng.optimizers.CMA

In [ ]:
iso3 = "PHL"
bcm = get_bcm_object(iso3, 'main')

In [ ]:
n_params = sum([p.size for p in bcm.priors.values()])
n_params

In [ ]:
opt = eng.optimize_model(bcm, obj_function=bcm.loglikelihood, num_workers=8, opt_class=opt_class)
res = opt.minimize(5000)
best_params = res.value[1]

In [ ]:
_ = plot_model_fit(bcm, best_params, iso3)

# Run multiple scenarios

In [ ]:
import pandas as pd
from autumn.projects.sm_covid2.common_school.project_maker import get_school_project

In [ ]:
project = get_school_project(iso3, 'main')
default_params = project.param_set.baseline
model = project.build_model(default_params.to_dict()) 

In [ ]:
output_names=['cumulative_incidence', 'cumulative_infection_deaths', 'ever_infected',
       'hospital_admissions', 'hospital_occupancy', 'incidence',
       'incidenceXagegroup_0', 'incidenceXagegroup_15',
       'incidenceXagegroup_25', 'incidenceXagegroup_50',
       'incidenceXagegroup_70', 'infection_deaths', 'infection_deaths_ma7',
       'n_immune_unvaccinated', 'n_immune_vaccinated',
       'peak_hospital_occupancy', 'prop_ever_infected',
       'prop_ever_infected_age_matched', 'prop_immune_unvaccinated',
       'prop_immune_vaccinated', 'student_weeks_missed', 'total_population',
       'transformed_random_process']

calibrated_params = default_params.update(best_params, calibration_format=True)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
df = model.get_derived_outputs_df()

In [ ]:
df.columns

In [ ]:
scenario_params = {
    "baseline": {},
    "scenario_1": {
        "mobility": {
            "unesco_partial_opening_value": 1.,
            "unesco_full_closure_value": 1.
        }
    },
}

outputs_df = pd.DataFrame(columns=output_names + ["scenario"])
for sc_name, sc_params in scenario_params.items():
    params = calibrated_params.update(sc_params)          
    model.run(params.to_dict())
    derived_df = model.get_derived_outputs_df()[output_names]
    derived_df["scenario"] = [sc_name] * len(derived_df)
    
    outputs_df = outputs_df.append(derived_df)

fig, ax = plt.subplots(figsize=(10,6))
for label, grp in outputs_df.groupby('scenario'):
    grp['transformed_random_process'].plot(ax=ax, label=label)

plt.legend()

In [ ]:
s = model.stratifications['agegroup']

In [ ]:
from copy import copy

In [ ]:
# colours = ["cornflowerblue", "darkorange", "mediumseagreen", "pink", "purple"]
colours = ["cornflowerblue", "slateblue", "mediumseagreen", "lightcoral", "purple"]

def plot_incidence_by_age(derived_outputs, ax, as_proportion: bool):
    # update_rcparams()
    y_label = "COVID-19 incidence proportion" if as_proportion else "COVID-19 incidence"    

    times = derived_outputs["incidence"].index.to_list()
    running_total = [0] * len(derived_outputs["incidence"])
    age_groups = [0, 15, 25, 50, 70]

    y_max = 1. if as_proportion else max([derived_outputs["incidence"].max() for sc in [0, 1]])

    for i_age, age_group in enumerate(age_groups):
        output_name = f"incidenceXagegroup_{age_group}"
    
        if i_age < len(age_groups) - 1:
            upper_age = age_groups[i_age + 1] - 1 if i_age < len(age_groups) - 1 else ""
            age_group_name = f"{age_group}-{upper_age}"
        else:
            age_group_name = f"{age_group}+"

        age_group_incidence = derived_outputs[output_name].to_list()
        
        if as_proportion:
            numerator, denominator = age_group_incidence, derived_outputs["incidence"].to_list()
            age_group_proportion = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
            new_running_total = age_group_proportion + running_total
        else: 
            new_running_total = [sum(x) for x in zip(age_group_incidence, running_total)]  


        ax.fill_between(times, running_total, new_running_total, color=colours[i_age], label=age_group_name, zorder=2, alpha=.8)
        running_total = copy(new_running_total)

    # y_max = max(new_running_total)
    plot_ymax = y_max * 1.1
    # add_school_closure_patches(ax, ISO3, ymax=plot_ymax)

    # work out first time with positive incidence
    t_min = derived_outputs['incidence'].gt(0).idxmax()    
    # ax.set_xlim((t_min, model_end))
    ax.set_ylim((0, plot_ymax))

    ax.set_ylabel(y_label)

    # if not as_proportion and scenario == 0:
    if True:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(
            reversed(handles),
            reversed(labels),
            # title="Age:",
            # fontsize=12,
            # title_fontsize=12,
            labelspacing=.2,
            handlelength=1.,
            handletextpad=.5,
            columnspacing=1.,
            facecolor="white",
            ncol=2,

        )

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(15,10))

top_ax = axes[0]
plot_incidence_by_age(outputs_df[outputs_df['scenario'] == "baseline"], ax=top_ax, as_proportion=True)

bottom_ax = axes[1]
plot_incidence_by_age(outputs_df[outputs_df['scenario'] == "scenario_1"], ax=bottom_ax, as_proportion=True)